In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_kidney_disease"
cohort = "GSE180393"

# Input paths
in_trait_dir = "../../input/GEO/Chronic_kidney_disease"
in_cohort_dir = "../../input/GEO/Chronic_kidney_disease/GSE180393"

# Output paths
out_data_file = "../../output/preprocess/Chronic_kidney_disease/GSE180393.csv"
out_gene_data_file = "../../output/preprocess/Chronic_kidney_disease/gene_data/GSE180393.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_kidney_disease/clinical_data/GSE180393.csv"
json_path = "../../output/preprocess/Chronic_kidney_disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Glomerular Transcriptome in the Cprobe Cohort"
!Series_summary	"We used microarrays to analyze the transcriptome of microdissected renal biopsies from patients with kidney disease and living donors. We derived pathway specific scores for Angiopoietin-Tie signaling pathway activation at mRNA level (or transcriptome level) for individual patients and studied the association of pathway activation with disease outcomes."
!Series_overall_design	"Glomerular gene expression data from micro-dissected human kidney biopsy samples  from patients with chronic kidney disease(Lupus, DN, IgA,HT, TN) and healthy living donors. Profiling was performed on Affymetrix ST2.1 microarray platform. "
Sample Characteristics Dictionary:
{0: ['sample group: Living donor', 'sample group: infection-associated GN', 'sample group: FSGS', 'sample group: LN-WHO III', 'sample group: LN-WHO IV', 'sample group: DN', 'sample group: amyloidosis', 'sample group: Membrano-Proliferative 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
from typing import Optional, Callable, Dict, Any
import json
import numpy as np

# 1. Gene Expression Data Availability
# Based on the background info, this dataset contains glomerular gene expression data from
# microarrays (Affymetrix ST2.1 platform), so it's suitable for our analysis
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From sample characteristics, we can see information about the disease status in row 0
trait_row = 0
# Age and gender are not explicitly mentioned in the sample characteristics
age_row = None
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value: str) -> int:
    """
    Convert the disease status to binary where:
    0 = Control/Healthy (Living donor)
    1 = CKD (any other condition)
    """
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Living donors are controls, all other conditions represent CKD
    if 'Living donor' in value:
        return 0
    else:
        return 1

def convert_age(value: str) -> Optional[float]:
    """
    Convert age value to float.
    Not used in this dataset as age information is not available.
    """
    return None

def convert_gender(value: str) -> Optional[int]:
    """
    Convert gender to binary where:
    0 = Female
    1 = Male
    Not used in this dataset as gender information is not available.
    """
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Save the initial filtering information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Try to find the clinical data file
    clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
    sample_char_path = os.path.join(in_cohort_dir, "sample_characteristics.txt")
    
    if os.path.exists(clinical_data_path):
        # If clinical_data.csv exists, read it directly
        clinical_data = pd.read_csv(clinical_data_path)
    elif os.path.exists(sample_char_path):
        # If sample_characteristics.txt exists, read it
        clinical_data = pd.read_csv(sample_char_path, sep='\t', index_col=0)
    else:
        # If neither file exists, create a dataframe from the sample characteristics dictionary
        # that was shown in the previous output
        
        # Sample values from the dictionary
        sample_groups = [
            'sample group: Living donor', 
            'sample group: infection-associated GN',
            'sample group: FSGS',
            'sample group: LN-WHO III',
            'sample group: LN-WHO IV',
            'sample group: DN',
            'sample group: amyloidosis',
            'sample group: Membrano-Proliferative GN',
            'sample group: MN',
            'sample group: AKI',
            'sample group: LN-WHO V',
            'sample group: FGGS',
            "sample group: 2'FSGS",
            'sample group: Thin-BMD',
            'sample group: Immuncomplex GN',
            'sample group: LN-WHO-V',
            'sample group: IgAN',
            'sample group: LN-WHO IV+V',
            'sample group: LN-WHO III+V',
            'sample group: LN-WHO-I/II',
            'sample group: chronic Glomerulonephritis (GN) with infiltration by CLL',
            'sample group: CKD with mod-severe Interstitial fibrosis',
            'sample group: Fibrillary GN',
            'sample group: Interstitial nephritis',
            'sample group: Hypertensive Nephrosclerosis',
            'sample group: Unaffected parts of Tumor Nephrectomy'
        ]
        
        # Create sample IDs (we don't have real IDs, so we'll use placeholders)
        sample_ids = [f"GSM{i+1}" for i in range(len(sample_groups))]
        
        # Create a DataFrame with rows for each sample and a column for the trait
        data = {
            0: sample_groups  # Using key 0 for 'sample group' as in the original dictionary
        }
        
        # Create the clinical data DataFrame with appropriate index
        clinical_data = pd.DataFrame(data, index=sample_ids)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the DataFrame
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{0: [0.0]}
Clinical data saved to ../../output/preprocess/Chronic_kidney_disease/clinical_data/GSE180393.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Assume gene data is available

# Extract gene data
try:
    # Extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # Print the first 20 gene/probe identifiers
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20].tolist())
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print(f"File path: {matrix_file}")
    print("Please check if the file exists and contains the expected markers.")


SOFT file: ../../input/GEO/Chronic_kidney_disease/GSE180393/GSE180393_family.soft.gz
Matrix file: ../../input/GEO/Chronic_kidney_disease/GSE180393/GSE180393_series_matrix.txt.gz
Gene data shape: (25582, 62)
First 20 gene/probe identifiers:
['100009613_at', '100009676_at', '10000_at', '10001_at', '10002_at', '100033413_at', '100033422_at', '100033423_at', '100033424_at', '100033425_at', '100033426_at', '100033427_at', '100033428_at', '100033430_at', '100033431_at', '100033432_at', '100033434_at', '100033435_at', '100033436_at', '100033437_at']


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers shown, these appear to be Affymetrix Mouse probe IDs 
# rather than standard human gene symbols. The format "number_at" is characteristic
# of Affymetrix arrays, and the presence of "at" suffix indicates this is likely 
# mouse data (Mouse430_2 array).
#
# These identifiers need to be mapped to standard gene symbols for analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Get a more complete view to understand the annotation structure
print("\nComplete sample of a few rows:")
print(gene_annotation.iloc[:3].to_string())

# Check if there are any columns that might contain gene information beyond what we've seen
potential_gene_columns = [col for col in gene_annotation.columns if 
                          any(term in col.upper() for term in ["GENE", "SYMBOL", "NAME", "ID"])]
print(f"\nPotential gene-related columns: {potential_gene_columns}")

# Look for additional columns that might contain gene symbols
# Since we only have 'ID' and 'ENTREZ_GENE_ID', check if we need to use Entrez IDs for mapping
gene_id_col = 'ID'
gene_symbol_col = None

# Check various potential column names for gene symbols
for col_name in ['GENE_SYMBOL', 'SYMBOL', 'GENE', 'GENE_NAME', 'GB_ACC']:
    if col_name in gene_annotation.columns:
        gene_symbol_col = col_name
        break

# If no dedicated symbol column is found, we'll need to use ENTREZ_GENE_ID
if gene_symbol_col is None and 'ENTREZ_GENE_ID' in gene_annotation.columns:
    gene_symbol_col = 'ENTREZ_GENE_ID'
    print("\nNo direct gene symbol column found. Will use Entrez Gene IDs for mapping.")

if gene_id_col in gene_annotation.columns and gene_symbol_col is not None:
    print(f"\nSample mappings from '{gene_id_col}' to '{gene_symbol_col}':")
    sample_mappings = gene_annotation[[gene_id_col, gene_symbol_col]].head(10)
    print(sample_mappings)
    
    # Check for non-null mappings to confirm data quality
    non_null_mappings = gene_annotation[[gene_id_col, gene_symbol_col]].dropna(subset=[gene_symbol_col])
    print(f"\nNumber of probes with gene ID mappings: {len(non_null_mappings)}")
    print(f"Sample of valid mappings:")
    print(non_null_mappings.head(5))
else:
    print("Required mapping columns not found in the annotation data. Will need to explore alternative mapping approaches.")



Gene annotation preview:
Columns in gene annotation: ['ID', 'ENTREZ_GENE_ID']
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'ENTREZ_GENE_ID': ['1', '10', '100', '1000', '10000']}

Complete sample of a few rows:
       ID ENTREZ_GENE_ID
0    1_at              1
1   10_at             10
2  100_at            100

Potential gene-related columns: ['ID', 'ENTREZ_GENE_ID']

No direct gene symbol column found. Will use Entrez Gene IDs for mapping.

Sample mappings from 'ID' to 'ENTREZ_GENE_ID':
             ID ENTREZ_GENE_ID
0          1_at              1
1         10_at             10
2        100_at            100
3       1000_at           1000
4      10000_at          10000
5  100009613_at      100009613
6  100009676_at      100009676
7      10001_at          10001
8      10002_at          10002
9      10003_at          10003

Number of probes with gene ID mappings: 1611728
Sample of valid mappings:
         ID ENTREZ_GENE_ID
0      1_at              1
1     10_at             

### Step 6: Gene Identifier Mapping

In [7]:
# Import necessary modules
import numpy as np
import pandas as pd
import os

# 1. Identify the appropriate keys for mapping in the gene annotation data
gene_id_col = 'ID'  # Column in gene_annotation for probe IDs
gene_symbol_col = 'ENTREZ_GENE_ID'  # Column in gene_annotation for gene identifiers

# 2. Get gene mapping dataframe by extracting the probe IDs and gene symbols
mapping_data = get_gene_mapping(gene_annotation, gene_id_col, gene_symbol_col)
print(f"Mapping data shape: {mapping_data.shape}")
print("First few rows of mapping data:")
print(mapping_data.head())

# We need to check if there's a mismatch between probe IDs in gene_data and mapping_data
gene_data_probes = set(gene_data.index)
mapping_probes = set(mapping_data['ID'])
common_probes = gene_data_probes.intersection(mapping_probes)
print(f"Number of probes in gene_data: {len(gene_data_probes)}")
print(f"Number of probes in mapping_data: {len(mapping_probes)}")
print(f"Number of common probes: {len(common_probes)}")

# Print sample probes from both datasets
print("Sample probe IDs from gene_data:")
print(list(gene_data.index[:5]))
print("Sample probe IDs from mapping_data:")
print(list(mapping_data['ID'][:5]))

# For this dataset, we need to use the Entrez gene IDs directly as gene symbols
# Create a modified mapping dataframe that contains only the probes in gene_data
filtered_mapping = mapping_data[mapping_data['ID'].isin(gene_data.index)].copy()
print(f"Number of matched probes after filtering: {len(filtered_mapping)}")

# Check if we have sufficient matches
if len(filtered_mapping) > 0:
    # Convert the Entrez IDs to strings to use as gene symbols
    filtered_mapping['Gene'] = filtered_mapping['Gene'].astype(str)
    
    # Apply gene mapping using the filtered mapping data
    gene_data = apply_gene_mapping(gene_data, filtered_mapping)
    print(f"Gene expression data shape after mapping: {gene_data.shape}")
    print("First few gene symbols after mapping:")
    print(list(gene_data.index[:10]))
else:
    print("ERROR: No matching probe IDs found between gene expression data and mapping data!")
    # Create an empty dataframe with the same columns as gene_data
    gene_data = pd.DataFrame(columns=gene_data.columns)

# Save the gene data file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Mapping data shape: (1611728, 2)
First few rows of mapping data:
         ID   Gene
0      1_at      1
1     10_at     10
2    100_at    100
3   1000_at   1000
4  10000_at  10000
Number of probes in gene_data: 25582
Number of probes in mapping_data: 25583
Number of common probes: 25582
Sample probe IDs from gene_data:
['100009613_at', '100009676_at', '10000_at', '10001_at', '10002_at']
Sample probe IDs from mapping_data:
['1_at', '10_at', '100_at', '1000_at', '10000_at']
Number of matched probes after filtering: 1611666


Gene expression data shape after mapping: (0, 62)
First few gene symbols after mapping:
[]
Gene expression data saved to ../../output/preprocess/Chronic_kidney_disease/gene_data/GSE180393.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Let's re-analyze the clinical data to ensure we're accessing the correct rows
background_info, clinical_data = get_background_and_clinical_data(matrix_file)
print("Clinical data shape:", clinical_data.shape)
print("Clinical data preview:")
print(clinical_data.head())

# Get a better understanding of the sample characteristics
sample_characteristics_dict = get_unique_values_by_row(clinical_data)
print("\nSample characteristics dictionary:")
print(sample_characteristics_dict)

# Based on our review of the sample characteristics, let's determine if we have trait data
is_trait_available = False
trait_row = None
age_row = None 
gender_row = None

# Check if sample_characteristics_dict contains data related to chronic kidney disease
for row_idx, values in sample_characteristics_dict.items():
    sample_values_str = str(values).lower()
    if any(term in sample_values_str for term in ['ckd', 'chronic kidney', 'kidney disease', 'renal']):
        trait_row = row_idx
        is_trait_available = True
        break

# Normalize gene data if it exists (from previous step)
try:
    gene_data = pd.read_csv(out_gene_data_file, index_col=0)
    print(f"Loaded gene data shape: {gene_data.shape}")
    
    if gene_data.shape[0] > 0:
        # Apply normalization only if gene data has content
        gene_data = normalize_gene_symbols_in_index(gene_data)
        print(f"Normalized gene data shape: {gene_data.shape}")
        is_gene_available = True
    else:
        print("Gene data is empty after mapping.")
        is_gene_available = False
except Exception as e:
    print(f"Error loading gene data: {e}")
    is_gene_available = False

# Save the normalized gene data
if is_gene_available and gene_data.shape[0] > 0:
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")

# If we have both trait and gene data, proceed with linking
if is_trait_available and is_gene_available and gene_data.shape[0] > 0:
    # Define conversion functions for clinical features
    def convert_trait(value):
        """
        Convert kidney disease status to binary:
        1 = has chronic kidney disease (any disease state)
        0 = healthy control (living donor)
        """
        if pd.isna(value):
            return None
        
        value = str(value).lower().strip()
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        if 'living donor' in value:
            return 0
        elif any(term in value for term in ['ckd', 'gn', 'nephritis', 'nephrosclerosis', 'fsgs', 'igan']):
            return 1
        else:
            return None
    
    # Extract clinical features
    try:
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=None,
            convert_age=None,
            gender_row=None,
            convert_gender=None
        )
        print(f"Extracted clinical data shape: {selected_clinical_df.shape}")
        print("Clinical data preview:")
        print(preview_df(selected_clinical_df))
        
        # Save clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
        
        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        
        # Handle missing values
        if linked_data.shape[0] > 0:
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            # Check for bias
            is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
            
            # Validate data quality
            note = "Dataset contains gene expression from kidney biopsies of patients with kidney disease and living donors."
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=is_biased,
                df=linked_data,
                note=note
            )
            
            # Save final linked data if usable
            if is_usable:
                os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                linked_data.to_csv(out_data_file)
                print(f"Linked data saved to {out_data_file}")
            else:
                print("Data not usable for the trait study - not saving final linked data.")
        else:
            print("Linked data is empty - cannot proceed with processing.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,  # Empty data is essentially biased
                df=pd.DataFrame(),  # Empty dataframe for metadata
                note="Failed to link clinical and genetic data - resulting dataset was empty."
            )
    except Exception as e:
        print(f"Error in clinical data processing: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort, 
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,
            df=pd.DataFrame(),
            note=f"Error in data processing: {str(e)}"
        )
else:
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable, so mark as biased
        df=pd.DataFrame(),  # Empty dataframe for metadata
        note="Missing essential data components for linking (trait data or gene expression data)."
    )

Clinical data shape: (2, 63)
Clinical data preview:
         !Sample_geo_accession                            GSM5607752  \
0  !Sample_characteristics_ch1            sample group: Living donor   
1  !Sample_characteristics_ch1  tissue: Glomeruli from kidney biopsy   

                             GSM5607753                            GSM5607754  \
0            sample group: Living donor            sample group: Living donor   
1  tissue: Glomeruli from kidney biopsy  tissue: Glomeruli from kidney biopsy   

                             GSM5607755                            GSM5607756  \
0            sample group: Living donor            sample group: Living donor   
1  tissue: Glomeruli from kidney biopsy  tissue: Glomeruli from kidney biopsy   

                             GSM5607757                            GSM5607758  \
0            sample group: Living donor            sample group: Living donor   
1  tissue: Glomeruli from kidney biopsy  tissue: Glomeruli from kidney biopsy   
